In [1]:
import pandas as pd
import sqlalchemy

In [2]:
fifa = pd.read_csv('resources/fifa20_data.csv', encoding='utf-8')
epl = pd.read_csv('resources/players_1920_fin.csv', encoding='utf-8')

/Users/troyramsey/opt/anaconda3/envs/UODataClass/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
epl.head()

,Unnamed: 0,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,full,team,ppm
0,1,0,0,1,0,0.1,534,37,1,0,...,1,0,0,0,4.5,False,0,Aaron Connolly,Brighton,0.222222
1,2,0,0,1,0,0.3,534,43,1,0,...,1,10589,13500,2911,4.5,True,0,Aaron Connolly,Brighton,0.222222
2,3,0,0,1,0,4.8,534,58,0,0,...,1,8090,11749,3659,4.5,False,0,Aaron Connolly,Brighton,0.222222
3,4,0,0,2,0,0.6,534,63,1,0,...,1,8437,13439,5002,4.5,False,0,Aaron Connolly,Brighton,0.222222
4,5,0,3,53,1,23.8,534,72,0,2,...,13,2156,5952,3796,4.5,True,0,Aaron Connolly,Brighton,2.888889


In [16]:
fifa.head()

,Name,Position,Country,Overall,Potential,Height,Value
0,Lionel Messi,"RW,CF,ST",Argentina,94,94,"5'7""",€95.5M
1,C. Ronaldo dos Santos Aveiro,"ST,LW",Portugal,93,93,"6'2""",€58.5M
2,Neymar da Silva Santos Jr.,"LW,CAM",Brazil,92,92,"5'9""",€105.5M
3,Jan Oblak,GK,Slovenia,91,91,"6'2""",€77.5M
4,Kevin De Bruyne,"CAM,CM",Belgium,91,91,"5'11""",€90M


# What to look for:

- EPL
    - Aggregate/group
    - goals and assists
    - red and yellow
    - Club
    
- FIFA
    - Age
    - Position
    - Overall
    - Potential
    - Country
    - Height
    - Value

In [5]:
epl = epl[['full', 'team','goals_scored', 'assists', 'yellow_cards', 'red_cards']]

In [6]:
fifa = fifa[['Name', 'Position', 'Country', 'Overall', 'Potential', 'Height', 'Value']]

In [21]:
import re

fifa_names = fifa['Name']
name_clean = epl.groupby('full')
name_check = pd.DataFrame(name_clean.sum()).reset_index()
broken_list = []
for i in name_check['full']:
    if '?' in i:
        broken_list.append(i)
broken_list
for i in broken_list:
    pattern = re.search(i.replace('?', '\S'))
    if pattern in 

['?aglar S?y?nc?',
 '?rjan Nyland',
 'Abdoulaye Doucour?',
 'Adalberto Pe?aranda',
 'Adama Traor?',
 'Adri?n San Miguel del Castillo',
 'Alexis S?nchez',
 'Andr? Filipe Tavares Gomes',
 'Antonio R?diger',
 'Ayoze P?rez',
 'Bernard An?cio Caldeira Duarte',
 'Borja Gonz?lez Tom?s',
 'Bruno Andr? Cavaco Jordao',
 'C?dric Soares',
 'C?sar Azpilicueta',
 'Carlos S?nchez',
 'Cheikhou Kouyat?',
 'Daniel Ceballos Fern?ndez',
 'Davinson S?nchez',
 'Davy Pr?pper',
 'Djibril Sidib?',
 'Emiliano Buend?a',
 'Emiliano Mart?nez',
 'Fabi?n Balbuena',
 'Fabian Sch?r',
 'Federico Fern?ndez',
 'Fr?d?ric Guilbert',
 'Francisco Femen?a Far',
 'Ga?tan Bong',
 'Georges-K?vin Nkoudou',
 'Gon?alo Bento Soares Cardoso',
 'H?ctor Beller?n',
 'H?lder Costa',
 'Ilkay G?ndogan',
 'Isma?la Sarr',
 'J?rgen Locadia',
 'Javier Hern?ndez Balc?zar',
 'Jes?s Vallejo L?zaro',
 'Jo?o Filipe Iria Santos Moutinho',
 'Jo?o Manuel Neves Virg?nia',
 'Jo?o Pedro Cavaco Cancelo',
 'Jo?o Pedro Junqueira de Jesus',
 'Joelinton C?ssi

TypeError: merge() missing 2 required positional arguments: 'left' and 'right'